In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup



Historical season betting data

In [47]:
hist_odds = pd.read_excel("nba odds 2020-21.xlsx")
hist_vist = hist_odds.iloc[::2]
hist_vist.index = range(533) 
hist_vist.loc[:, 'VH'] = 0
hist_vist['Team_V'] = hist_vist.loc[:, 'Team']
hist_vist['Final_V'] = hist_vist.loc[:,'Final']
hist_vist['O/U'] = hist_vist.loc[:,'Open']
hist_vist['ML_V'] = hist_vist.loc[:,'ML']
hist_vist = hist_vist[['VH', 'Team_V', 'Final_V', 'O/U', 'ML_V']]

hist_home = hist_odds.iloc[1::2]
hist_home.index = range(533) 
hist_home.loc[:, 'VH'] = 1
hist_home['Team_H'] = hist_home.loc[:, 'Team']
hist_home['Final_H'] = hist_home.loc[:,'Final']
hist_home['line'] = hist_home.loc[:, 'Open']
hist_home['ML_H'] = hist_home.loc[:, 'ML']
hist_home = hist_home[['VH', 'Team_H', 'Final_H', 'line', 'ML_H']]

hist_odds = pd.merge(left = hist_vist, right = hist_home, left_index = True, right_index = True, how = "outer")
hist_odds[hist_odds['O/U'] == 'pk'] = 0
hist_odds[hist_odds['line'] == 'pk'] = 0
hold1 = hist_odds['O/U']
hold2 = hist_odds['line']
hist_odds['O/U'] = [i if abs(i) > abs(j) else j for i,j in zip(hold1,hold2)]
hist_odds['line'] = [i if abs(i) < abs(j) else j for i,j in zip(hold1,hold2)]

line_sign = [-1 if x >= 0 else 1 for x in hist_odds['ML_H']]
hist_odds['line'] = hist_odds['line'] * line_sign


     VH_x        Team_V  Final_V    O/U  ML_V  VH_y      Team_H  Final_H  \
0       0   GoldenState       99  228.0   265     1    Brooklyn      125   
1       0    LAClippers      116  223.0   115     1    LALakers      109   
2       0     Charlotte      114  215.0  -140     1   Cleveland      121   
3       0       NewYork      107  214.5   270     1     Indiana      121   
4       0         Miami      107  220.5  -190     1     Orlando      113   
..    ...           ...      ...    ...   ...   ...         ...      ...   
528     0  OklahomaCity      107  218.5   230     1  SanAntonio      102   
529     0         Miami      103  225.0  -130     1  NewOrleans       93   
530     0    Sacramento      119  238.0   160     1    Portland      123   
531     0       Detroit      104  211.0   245     1     NewYork      114   
532     0   GoldenState       98  223.0   550     1     Phoenix      120   

     line  ML_H  
0     5.5  -320  
1     2.5  -135  
2    -1.0   120  
3     8.5  -330

C:\Users\thyang\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\thyang\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\thyang\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Today's lines

In [64]:
URL = 'https://www.scoresandodds.com/nba'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
ml = soup.find_all('span', class_ = 'data-moneyline')
money_line = [int(line.text) for line in ml]
team_name = soup.find_all('span', class_ = 'team-name')
teams = [name.text[0:(len(name.text)-5)] for name in team_name]
over = soup.find_all('td', class_ = 'tbd')
for o in over:
    o\u = o.find('data-value')
print(over)

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

Bet Sizing

In [ ]:
init_money = 100
prob_dif = pred_prob - bet_prob
for prob_dif > .3:
    kelly = pred_prob - (1-pred_prob)/bet_prob
    bet = kelly * init_money
    bet_win = (1+ bet_prob) * outcome * bet
    init_money = bet_win + bet

